In [ ]:
import pandas as pd
import concurrent.futures 
import snscrape.modules.twitter as snscrape
from datetime import datetime

# Tweet Collection 1
## Tweet Collection

Pengumpulan data dimulai dengan mengumpulkan tweets yang berkaitan dengan pemilu. SNScrape digunakan karena kemudahan implementasinya dan ketidakterkaitanya dengan API Twitter. 

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from datetime import datetime


maxTweets = 1000000
count = 0

COLUMN_NAME = ["UserID","User","Location","verified","Date_Created","Follows_Count","Friends_Count", 
        "Retweet_Count","Language","Date_Tweet","Number_of_Likes","Source_of_Tweet",
        "Tweet_Id","Tweet","Hashtags","Conversation_Id","In_reply_To","Latitude","Longitude","Place"]
df = pd.DataFrame(columns=COLUMN_NAME)

keyword = "#Pilpres2019 OR #DebatPilpres2019 OR #Pemilu2019 OR #BeraniPilih02 OR #JokowiMenangTotalDebat OR #DebatPintarJokowi OR #PrabowoMenangDebat OR #DebatKeduaPilpres2019 OR #Debat02PrabowoMenang OR #DebatCapres OR #PrabowoSandi OR #01JokowiLagi OR #2019PilihJokowi OR #2019GantiPresiden OR #01IndonesiaMaju OR #2019TetapJokowi OR #JokowiLagi OR #2019PrabowoSandi"
date = 'since:2018-09-01 until:2019-05-31 min_faves:1 exclude:retweets'

try: 
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(keyword + ' ' + date).get_items()):
        if count > maxTweets:
            break
        df.loc[i] = [
                                    tweet.user.id,
                                    tweet.user.username,
                                    tweet.user.location,
                                    tweet.user.verified,
                                    tweet.user.created,
                                    tweet.user.followersCount,
                                    tweet.user.friendsCount,
                                    tweet.retweetCount,
                                    tweet.lang,
                                    tweet.date,
                                    tweet.likeCount,
                                    tweet.sourceLabel,
                                    tweet.id,
                                    tweet.content,
                                    tweet.hashtags,
                                    tweet.conversationId,
                                    tweet.inReplyToUser,
                                    tweet.coordinates.latitude if tweet.coordinates != None else None,
                                    tweet.coordinates.longitude if tweet.coordinates != None else None,
                                    tweet.Place.fullName if tweet.place != None else None]
        count += 1
        print(count)

    date = datetime.now().strftime('%Y-%m-%d_%H_%M_%S')
    df.to_json(r'Pemilu_' + date + '.json', orient='records', lines=True)
    df.to_csv(r'Pemilu_' + date + '.csv', index=False)
except KeyboardInterrupt:
    print("KeyboardInterrupt")
    date = datetime.now().strftime('%Y-%m-%d_%H_%M_%S')
    df.to_json(r'Pemilu_Interrupt' + date + '.json', orient='records', lines=True)
    df.to_csv(r'Pemilu_Interrupt' + date + '.csv', index=False)






Kode diatas merupakan kode awal yang menggunakan SNScrape dalam mengumpulkan data.  
Parameter yang digunakan diantara lain:
1. Tanggal pengumpulan data yaitu 1 September 2018 hingga 1 Mei 2019
2. Min_faves merupakan like paling sedikit yang diperlukan
3. Exclude retweets dan replies dimana retweets dan replies tidak diikutsertakan dalam pengumpulan data  
---
Keyword yang digunakan merupakan hasil dari riset Twitter yang dipublikasikan dalam [Blog Twitter](https://blog.twitter.com/in_id/topics/events/2019/124-juta-tweet-seputar-pemilihan-umum-2019)  
Keyword tersebut merupakan:
1. #Pilpres2019
2. #DebatPilpres2019
3. #Pemilu2019
4. #BeraniPilih02
5. #JokowiMenangTotalDebat
6. #DebatPintarJokowi
7. #PrabowoMenangDebat
8. #DebatKeduaPilpres2019 
9. #Debat02PrabowoMenang 
10. #DebatCapres 
11. #PrabowoSandi 
12. #01JokowiLagi 
13. #2019PilihJokowi 
14. #2019GantiPresiden 
15. #01IndonesiaMaju 
16. #2019TetapJokowi 
17. #JokowiLagi 
18. #2019PrabowoSandi
---
Hanya 18 keyword tersebut yang dapat dipakai karena keyword dapat digunakan limit library SNScrape.  
Pengumpulan data menghasilkan data dengan 418931 tweets.  
Hasil Program tersebut merupakan file csv dan json yang dapat digunakan untuk analisis lebih lanjut.  

Analisis selanjutnya menentukan lokasi dari tweet tersebut. Lokasi akan ditentukan oleh kolom "Place" atau "Location" pada dataset tersebut.  
Namun, karena kolom tersebut tidak standar dan kadang berisi informasi yang tidak jelas atau tidak terisi sama sekali, maka filtering dilakukan untuk menentukan lokasi mereka.

### Top 20 Lokasi Tweets sebelum cleaning
| Lokaksi                        | Count |
|--------------------------------|-------|
| DKI Jakarta, Indonesia         | 11801 |
| Jakarta, Indonesia             | 9792  |
| Jakarta                        | 8536  |
| Jakarta Capital Region, Indone | 4282  |
| Kota Surabaya, Jawa Timur      | 3556  |
| Jakarta Capital Region         | 3292  |
| Bandung, Jawa Barat            | 3024  |
| Yogyakarta, Indonesia          | 2626  |
| Jakarta Selatan, DKI Jakarta   | 2279  |
| Jawa Barat, Indonesia          | 2133  |
| Jakarta Pusat, DKI Jakarta     | 2109  |
| indonesia                      | 2054  |
| Aceh                           | 1944  |
| Sumatera Barat, Indonesia      | 1798  |
| Jawa Tengah, Indonesia         | 1788  |
| Kota Medan, Sumatera Utara     | 1731  |
| INDONESIA                      | 1652  |
| Jakarta Timur, DKI Jakarta     | 1538  |
| Denpasar Selatan, Indonesia    | 1382  |
| mojokerto                      | 1104  |

## Filtering via Kabutapen Kota
Dapat dilihat pada data diatas bahwa lokasi yang diberikan tidak standar. Selain itu, terdapat lokasi yang standard seperti "indonesia" dan "INDONESIA".  
Didasari dengan lokasi yang tidak tentu, filtering dilakukan untuk menentukan lokasi dari user tersebut.  
Filtering dilakukan dengan mengecek kolom "Location" dengan data kabupaten kota yang ada di Indonesia.  
Jika ditemukan, maka lokasi tersebut akan dianggap sebagai lokasi user.  


In [ ]:
# kabupaten_kota merupakan file python yang berisi dictionary dari seluruh provinsi dan kabupaten/kota di Indonesia
import kabupaten_kota as kk

locations = df['Location'].tolist()

def check_kabupaten_kota():
    kabupaten_kota_list = []
    for loc in locations:
        flag = False
        # Jika lokasi tidak diketahui, maka akan dianggap sebagai "Unknown"
        if loc == "":
            kabupaten_kota_list.append("Unknown")
        
        # Jika lokasi mengandung nama kabupaten kota, maka akan dianggap sebagai nama kabupaten kota
        for prov in kk.provinsi:
            for kabkot in kk.kabupaten_kota[prov]:
                if kabkot in  str(loc).lower():
                    kabupaten_kota_list.append(kabkot)
                    flag = True
                    break
        if flag == False:
            kabupaten_kota_list.append("Unknown")
        
    return kabupaten_kota_list

kabupaten_kota_list = check_kabupaten_kota()


### Top 20 Kabupaten Kota Tweets setelah filtering
| Kabupaten Kota | Count  |
|----------------|--------|
| Unknown        | 285919 |
| jakarta        | 58317  |
| bandung        | 8550   |
| bekasi         | 4820   |
| surabaya       | 4718   |
| bogor          | 4430   |
| medan          | 4002   |
| yogyakarta     | 3747   |
| denpasar       | 2487   |
| tangerang      | 2394   |
| banjar         | 2015   |
| semarang       | 1926   |
| malang         | 1804   |
| depok          | 1723   |
| makassar       | 1642   |
| sidoarjo       | 1296   |
| padang         | 1285   |
| bukittinggi    | 1193   |
| palembang      | 1187   |
| mojokerto      | 1104   |

Dari total 418931 tweets, 133012 tweets berhasil diidentifikasi lokasinya.  
Selain itu, perbedaan jumlah tweets yang diidentifikasi per kabupaten kota juga drastis.  
Oleh karena itu, dilakukan filtering lagi untuk menentukan tweets yang didapatkan per provinsi.  

## Filtering via Provinsi
Metode filtering yang dilakukan sama, namun data yang digunakan adalah data provinsi.

In [ ]:
import kabupaten_kota as kk 

locations = df['Location'].tolist()

def check_provinsi():
    kabupaten_kota_list = []
    for loc in locations:
        flag = False
        # Jika lokasi tidak diketahui, maka akan dianggap sebagai "Unknown"
        if loc == "":
            kabupaten_kota_list.append("Unknown")
        # Jika lokasi mengandung nama provinsi, maka akan dianggap sebagai nama provinsi
        for prov in kk.provinsi:
            if prov in str(loc).lower():
                kabupaten_kota_list.append(prov)
                flag = True
                break
            # Jika lokasi mengandung nama kabupaten kota, maka akan dianggap sebagai nama provinsi
            for kabkot in kk. kabupaten_kota[prov]:
                if kabkot in  str(loc).lower():
                    kabupaten_kota_list.append(prov)
                    flag = True
                    break
        if flag == False:
            kabupaten_kota_list.append("Unknown")


provinsi_list = check_provinsi()

### Top 20 Provinsi Tweets setelah filtering
| Provinsi            | Count  |
|---------------------|--------|
| Not Specified       | 112305 |
| dki jakarta         | 56910  |
| jawa barat          | 25723  |
| jawa timur          | 15711  |
| jawa tengah         | 7866   |
| sumatera barat      | 4875   |
| sumatera utara      | 4817   |
| aceh                | 4742   |
| banten              | 4533   |
| bali                | 4261   |
| di yogyakarta       | 4151   |
| sulawesi selatan    | 2579   |
| riau                | 2454   |
| nusa tenggara barat | 2212   |
| lampung             | 1715   |
| kalimantan selatan  | 1710   |
| sumatera selatan    | 1430   |
| sulawesi utara       | 1087   |
| papua               | 1054   |

Kesimpulan sama terjadi pada filtering ini dimana jumlah tweets yang diidentifikasi per provinsi juga berbeda drastis.    
Ide lain dengan menggunakan commenter atau follower dari akun politik pada kabupaten / kota dilontarkan untuk mencoba mendapatkan data yang lebih banyak.  
Tetapi, menemukan akun politik yang memiliki follower atau commenter yang banyak juga tidak mudah.  
Disini juga diputuskan bahwa scope dari penelitian hanya sampai di pulau Jawa

## Tweet Collection 2

Untuk memperbesar dataset, scraping kembali dilakukan dengan menggunakan keyword yang sama. Untuk mempercepat proses pengumpulan tweets, algoritma yang digunakan dimodifikasi agar menggunakan multithreading.  

Digunakan module concurrent.futures untuk melakukan multithreading. Setelah dilakukan pengumpulan data, didapatkan 3965628 tweets dan 2847038 tweets yang memiliki lokasi.

Karena hasil yang masih imbalance, lokasi user unique yang memiliki lokasi digunakan untuk membuat dataset baru yang berisi dengan tweets yang memiliki di pulau jawa tersebut.  
Dengan menggunakan fungsi check_provinsi diatas, hasil yang didapatkan adalah 13616 user yang memiliki lokasi di pulau jawa.  
## Contoh hasil check_provinsi

| Username         | Provinsi    |
|------------------|-------------|
| warsahidayat     | jawa barat  |
| Hasanudin        | jawa barat  |
| serujinews       | jawa timur  |
| detikcom         | dki jakarta |
| okezonenews      | dki jakarta |
| banu_sukamto     | dki jakarta |
| netgrit          | dki jakarta |
| inewsdotid       | dki jakarta |
| dialogikapod     | dki jakarta |
| fadjroeL         | dki jakarta |
| hariankompas     | dki jakarta |
| mahadewa         | dki jakarta |
| HansiPenjaga     | jawa barat  |
| bambang_samekto  | jawa tengah |
| Tarikh_Tambang   | jawa timur  |
| MataNajwa        | dki jakarta |
| hakasasiid       | dki jakarta |
| kunartomarzuki   | jawa tengah |
| SeputariNews     | dki jakarta |
| mad_pecci        | jawa barat  |


## Filtering tweets
Karena tweets yang dibuat oleh user tidak selalu berkaitan dengan politik, maka dilakukan filtering dengan menggunakan keyword yang sama dengan scraping.  
Keyword dibuat dengan menggunakan sample 100000 tweets yang diambil dari dataset yang sudah dikumpulkan.

In [ ]:
import pandas as pd
from datetime import datetime
import random
import re,emoji



def find_url(string):
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    # regex untuk mencari seluruh url pada string
    url = re.findall(regex,string)
    # mengembalikan list url
    return [x[0] for x in url]

def get_word_count(sample):

    lexicon = {}
    for i in sample:
        # menghapus emoji
        i = emoji.replace_emoji(i,'').lower()
        for word in i.split():
            # Mengganti username menjadi @USER
            if word[0] == '@' or word == '[username]':
                word = '@USER'
            # Mengganti url menjadi HTTPURL
            elif find_url(word) != []:
                word = 'HTTPURL'
            elif word == 'httpurl' or word == '[url]':
                word = 'HTTPURL'
            
            # Menghapus tanda baca
            word = re.sub(r'[^\w\s]','',word)

            # menambahkan kata ke lexicon
            if (word != 'HTTPURL' or word != 'USER') and word not in lexicon:
                lexicon[word] = 1
            else:
                lexicon[word] += 1
    return lexicon 


df = pd.read_csv('Data/2022-11-12_00_21_00dropped.csv')
tweets = df['content'].tolist()
sampling = random.sample(tweets, 100000)
lexicon = get_word_count(sampling)

Lalu data akan dicek manual untuk menentukan apakah kata tersebut berkaitan dengan politik atau tidak.  
Contoh data yang dihasilkan dari filtering adalah sebagai berikut:

| text                | count  | keyword |
|---------------------|--------|---------|
| prabowobersamaulama | 181    | 1       |
| bangsa              | 1043   | 1       |
| nasional            | 204    | 1       |
| dpr                 | 280    | 1       |
| 1                   | 1114   | 1       |
| USER                | 372376 | 0       |
| yg                  | 23425  | 0       |
| HTTPURL             | 21844  | 0       |
| NaN                 | 18236  | 0       |


## Data Collection 3 dengan scope Pulau Jawa dan filtering menggunakan keyword

Scraping kembali dilakukan untuk memperbanyak tweets yang didapatkan. Pengumpulan tweets dilakukan kedua hasil filtering diatas.  
Data tersebut didapatkan dari user unique yang didapatkan dari data sebelumnya.  

In [ ]:
import pandas as pd
import concurrent.futures 
import snscrape.modules.twitter as snscrape
from datetime import datetime


words = pd.read_csv('keyword.csv', delimiter=';', encoding = "ISO-8859-1")
words = words[words['keyword'] == 1]

filtered_words = words['text'].tolist()

def filter_data(text):
    # Jika tweet mengandung kata kunci, maka akan dianggap sebagai data yang relevan
    if any(word in text for word in filtered_words):
        return True


def get_tweets(username):
    results = []
    username = username.strip()

    # parameter untuk pencarian tweet
    keyword = "#from:" + username
    date = 'since:2018-09-01 until:2019-05-31 min_faves:1 exclude:retweets'

    for i,tweet in enumerate(snscrape.TwitterSearchScraper(keyword + " " + date).get_items()):
        # Jika tweet mengandung kata kunci, maka akan dianggap sebagai data yang relevan
        if (filter_data(tweet.content)):
            results.append([username, tweet.user.location, tweet.id, tweet.content, tweet.renderedContent, tweet.inReplyToTweetId, tweet.inReplyToUser, tweet.likeCount, tweet.retweetCount, tweet.quoteCount, tweet.replyCount, tweet.date])
        # Jika sudah mendapatkan 2000 tweet, maka akan berhenti
        if (i == 2000):
            break
    print("Done getting tweets from ", username)
    return results


def concurrent_get_tweets(usernames):
    count = 0
    result = []
        
    # Inisialisasi thread pool dengan 6 thread
    with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
        # Memulai thread untuk setiap username
        futures = [executor.submit(get_tweets, username) for username in usernames]
        # Inisialisasi dataframe untuk autosave
        autosave = pd.DataFrame(columns=['username', 'location','tweetID', 'content', 'renderedContent', 'inReplyToTweetId', 'inReplyToUser', 'likeCount', 'retweetCount', 'quoteCount', 'replyCount', 'date'])
        # Menunggu semua thread selesai
        for future in concurrent.futures.as_completed(futures):
            try:
                # Mengambil hasil dari thread
                data = future.result()
                result.extend(data)
                count += 1
                # Autosave setiap 100 user
                if (count % 100 == 0):   
                    print("Done getting ", count, " out of ", len(usernames) , " users")
                    autosave = pd.DataFrame(result, columns=['username', 'location','tweetID', 'content', 'renderedContent', 'inReplyToTweetId', 'inReplyToUser', 'likeCount', 'retweetCount', 'quoteCount', 'replyCount', 'date'])
                    autosave.to_csv('autosave_jawa.csv', index=False)
                    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    print("Autosaved at ", now)
            except Exception as exc:
                print(exc)
                pass
    return result

Didapatkan tweets sebanyak 1943849 tweets dari 13616 user yang memiliki lokasi di pulau jawa.  
Setiap user memiliki dapat menambahkan maksimal 2000 tweets pada dataset.
## Pembagian provinsi hasil scraping

| Provinsi    | Count   |
|-------------|---------|
| dki jakarta | 1067556 |
| jawa barat  | 422217  |
| jawa timur  | 291467  |
| jawa tengah | 162609  |

Berdasarkan hasil tersebut, undersampling digunakan untuk menyeimbangkan data.

## Sampling
Hasil akhir sampling adalah 600000 tweets dengan 150000 tweets per provinsi.
| Provinsi    | Count  |
|-------------|--------|
| dki jakarta | 150000 |
| jawa barat  | 150000 |
| jawa tengah | 150000 |
| jawa timur  | 150000 |

In [ ]:
nMax = 150000

sampled = df.groupby('provinsi').apply(lambda x: x.sample(n=min(nMax, len(x))))